In [1]:
import pika
import uuid

In [2]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)
exchangeName = 'direct_pings'

In [3]:
!sudo rabbitmqctl list_exchanges

Listing exchanges ...
	direct
amq.direct	direct
amq.fanout	fanout
amq.headers	headers
amq.match	headers
amq.rabbitmq.log	topic
amq.rabbitmq.trace	topic
amq.topic	topic
direct_pings	direct


In [4]:
!sudo rabbitmqctl list_bindings

Listing bindings ...
	exchange	amq.gen-KgXlN-n7LIPky1SpfrrRoQ	queue	amq.gen-KgXlN-n7LIPky1SpfrrRoQ	[]
direct_pings	exchange	amq.gen-KgXlN-n7LIPky1SpfrrRoQ	queue	ping_queue	[]


Below we will create a connection, channel, and queue

**queue_declare(queue, passive=False, durable=False, exclusive=False, auto_delete=False, arguments=None, callback=None)**<br>
<br>
Declare queue, create if needed. This method creates or checks a queue. When creating a new queue the client can specify various properties that control the durability of the queue and its contents, and the level of sharing for the queue.

**basic_consume(queue, on_message_callback, auto_ack=False, exclusive=False, consumer_tag=None, arguments=None, callback=None)**

Sends the AMQP 0-9-1 command Basic.Consume to the broker and binds messages for the consumer_tag to the consumer callback. If you do not pass in a consumer_tag, one will be automatically generated for you. Returns the consumer tag.

**queue (str)** – The queue to consume from. Use the empty string to specify the most recent server-named queue for this channel

**on_message_callback**: (callable) –
The function to call when consuming with the **signature on_message_callback(channel, method, properties, body)**, where

* **channel**: pika.Channel 

* **method**: pika.spec.Basic.Deliver 

* **properties**: pika.spec.BasicProperties 

* **body**: bytes


### Message properties

The AMQP 0-9-1 protocol predefines a set of 14 properties that go with a message. Most of the properties are rarely used, with the exception of the following:

* delivery_mode: Marks a message as persistent (with a value of 2) or transient (any other value). 
* content_type: Used to describe the mime-type of the encoding. For example for the often used JSON encoding it is a good practice to set this property to: application/json.
* reply_to: Commonly used to name a callback queue.
* correlation_id: Useful to correlate RPC responses with requests.

**See https://www.rabbitmq.com/tutorials/tutorial-six-python.html for details**

In [5]:
class TalkToWorkersClient(object):

    def __init__(self):
        #self.connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))

        self.connection = pika.BlockingConnection(parameters)
        self.channel = self.connection.channel()
        self.channel.exchange_declare(exchange=exchangeName, exchange_type='direct')
        
        self.ping_responses=[]
        
        myqueue = self.channel.queue_declare('', exclusive=True)
        print("myqueue:", myqueue)
        
        self.callback_queue = myqueue.method.queue #The queue we just created

        print("myqueue.method:", myqueue.method)
        
        # Let us start waiting for messages (The results of our call). 
        #When we recevie a message, we will the on_response call back function
        self.channel.basic_consume(
            queue=self.callback_queue, #the queue we created above
            on_message_callback=self.on_response, #The callback function to run when we receive a message from the workers
            auto_ack=True)
        

    def on_response(self, ch, method, props, body):
        if self.corr_id == props.correlation_id: #Check if our correlation id is the same as we get in the message
            self.response = body
            self.ping_responses.append(body)
            print(self.response)

    def send_task2worker(self, worker_routing):
        self.response = None
        self.corr_id = str(uuid.uuid4()) # This is where we create our unique correlation id
        print('corr_id:', self.corr_id)
        
        # Now we are ready to send our message to the workers
        self.channel.basic_publish(
            exchange=exchangeName,
            routing_key='rpc_queue',
            properties=pika.BasicProperties(
                reply_to=self.callback_queue,
                correlation_id=self.corr_id,
                delivery_mode =1,
                content_type ="text/plain"
            ),
            body=str(n))
        
        #Waiting for the results
        while self.response is None:
            self.connection.process_data_events()
        return self.response
        
    def ping_workers(self):
        self.response = None
        self.corr_id = str(uuid.uuid4()) # This is where we create our unique correlation id
        print('corr_id:', self.corr_id)
        
        # Now we are ready to send our message to the workers
        self.channel.basic_publish(
            exchange=exchangeName,
            routing_key='ping_queue',
            properties=pika.BasicProperties(
                reply_to=self.callback_queue,
                correlation_id=self.corr_id,
                delivery_mode =1,
                content_type ="text/plain"
            ),
            body='Ping!')
        
        #Waiting for the results
        while self.response is None:
            self.connection.process_data_events()
        return self.response
    

In [6]:
#This will create one channel and queue. All communications will be done on that queue using different correlation ids. 
talkToWorkersClient_instance = TalkToWorkersClient()

myqueue: <METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=amq.gen-G4z2TFQiKK1GC-RX92H1Lg'])>"])>
myqueue.method: <Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=amq.gen-G4z2TFQiKK1GC-RX92H1Lg'])>


In [7]:
response = talkToWorkersClient_instance.ping_workers()
print(response)

corr_id: 727b42d5-0083-4a71-ac7e-b08b4c20ade7
b'routing_172.31.14.37_10936'
 [.] Got b'routing_172.31.14.37_10936'


In [9]:
!sudo rabbitmqctl list_bindings

Listing bindings ...
	exchange	amq.gen-G4z2TFQiKK1GC-RX92H1Lg	queue	amq.gen-G4z2TFQiKK1GC-RX92H1Lg	[]
	exchange	amq.gen-KgXlN-n7LIPky1SpfrrRoQ	queue	amq.gen-KgXlN-n7LIPky1SpfrrRoQ	[]
direct_pings	exchange	amq.gen-KgXlN-n7LIPky1SpfrrRoQ	queue	ping_queue	[]
direct_pings	exchange	amq.gen-KgXlN-n7LIPky1SpfrrRoQ	queue	routing_172.31.14.37_10936	[]


In [10]:
talkToWorkersClient_instance.ping_responses

[b'routing_172.31.14.37_10936']

In [12]:
response = talkToWorkersClient_instance.ping_workers()
print(response)

corr_id: 2bfccbe1-523b-42dc-af35-76111fdebe6c


StreamLostError: Stream connection lost: ConnectionResetError(104, 'Connection reset by peer')

In [13]:
talkToWorkersClient_instance.ping_responses

[b'routing_172.31.14.37_10936']

In [8]:
print(" [x] sending message")
response = fibonacci_rpc.call('huh')
print(" [.] Got %r" % response)

 [x] sending message
corr_id: 34aeae03-2d2b-4729-94aa-35f45007410e
b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 5449 time: 1568681518.1953926 message: b'huh'"
b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 6634 time: 1568681518.1957557 message: b'huh'"
 [.] Got b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 6634 time: 1568681518.1957557 message: b'huh'"


In [9]:
fibonacci_rpc.ping_responses

[b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 6634 time: 1568681517.4342203 message: b'hi'",
 b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 5449 time: 1568681517.4351234 message: b'hi'",
 b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 5449 time: 1568681518.1953926 message: b'huh'",
 b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 6634 time: 1568681518.1957557 message: b'huh'"]

In [10]:
print(" [x] sending message")
response = fibonacci_rpc.call('mehh')
print(" [.] Got %r" % response)

 [x] sending message
corr_id: e6d91786-bc1a-4190-a555-872ba93ce107
b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 6634 time: 1568681583.291425 message: b'mehh'"
b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 5449 time: 1568681583.291952 message: b'mehh'"
 [.] Got b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 5449 time: 1568681583.291952 message: b'mehh'"


In [19]:
response = fibonacci_rpc.call('hmm')
print(" [.] Got %r" % response)

corr_id: bfffb612-b740-4318-8739-8566654ed953
b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 6634 time: 1568681674.904041 message: b'hmm'"
b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 5449 time: 1568681674.9215417 message: b'hmm'"
 [.] Got b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 5449 time: 1568681674.9215417 message: b'hmm'"


In [20]:
fibonacci_rpc.ping_responses

[b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 6634 time: 1568681517.4342203 message: b'hi'",
 b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 5449 time: 1568681517.4351234 message: b'hi'",
 b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 5449 time: 1568681518.1953926 message: b'huh'",
 b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 6634 time: 1568681518.1957557 message: b'huh'",
 b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 6634 time: 1568681583.291425 message: b'mehh'",
 b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 5449 time: 1568681583.291952 message: b'mehh'",
 b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 6634 time: 1568681585.5706172 message: b'lalala'",
 b"172.31.14.37 process_name: MainProcess process_index: _none_ os_pid: 6634 time: 1568681626.9531314 message: b'lalala'",
 b"172.31.14.37 process_name: MainPr

In [21]:
response = fibonacci_rpc.call('hmm :)')
print(" [.] Got %r" % response)

corr_id: 78ed9a32-c64a-4b17-b03a-7c0b6ea3648b


StreamLostError: Stream connection lost: ConnectionResetError(104, 'Connection reset by peer')